In [ ]:
import os
import torch
import numpy as np
import imageio
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from time import time as time

from sklearn.metrics import confusion_matrix
import torchnet as tnt
import functools

import mock
from tqdm import tqdm_notebook as tqdm


# GLOBAL SETTINGS
PlotSize = 8                                     # Size of plots
matplotlib.rcParams['figure.figsize'] = [PlotSize*2, PlotSize]  
CMAP = matplotlib.colors.ListedColormap(['black', 'white', 'orange'])               # Color mapping 
np.set_printoptions(precision=2, suppress=True)  # Array print precision

# CLASS AND FEATURE DESCRIPTION
class_names = ['BACKGRD','PINUS','PICEA']

# PATHS TO TRAIN/TEST DATA
data_path = '../data/split_05__05/'
training_set_path = data_path + 'train/'         # Relative path to training patch root folder
test_set_path =     data_path + 'test/'          # Relative path to test patch root folder

num_of_training_tiles = len(os.listdir(training_set_path + 'GT/'))
num_of_test_tiles = len(os.listdir(test_set_path + 'GT/'))

# USE CIR OR RGB DATA
use_cir = True
use_rgb = True

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

print(torch.cuda.get_device_name())
print(torch.cuda.get_device_capability())
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

In [ ]:
def read_patch(root_folder, cir=True, rgb=True):
    ##########################################################
    # READ IMAGES as FLOAT
    
    if cir:
        cir_file_list = os.listdir(root_folder + 'CIR/')
        cir_list = []
        
        for file in cir_file_list:
            cir_patch = imageio.imread(root_folder + 'CIR/' + file).astype(np.float32)
    
            cir_list.append(cir_patch[:,:,:].transpose([2,0,1]))
            del cir_patch

        cir_features = np.stack(cir_list, axis=0)
    
    
    if rgb:
        rgb_file_list = os.listdir(root_folder + 'RGB/')
        rgb_list = []
        
        for file in rgb_file_list:
            rgb_patch = imageio.imread(root_folder + 'RGB/' + file).astype(np.float32)
    
            rgb_list.append(rgb_patch[:,:,:].transpose([2,0,1]))
            del rgb_patch
        
        rgb_features = np.stack(rgb_list, axis=0)


    gt_file_list = os.listdir(root_folder + 'GT/')
    gt_list = []

    for file in gt_file_list:
        gt_patch = imageio.imread(root_folder + 'GT/' + file).astype(np.int64)
 
        gt_list.append(gt_patch[:,:])
        del gt_patch

    
    if cir and rgb:
        features = np.concatenate([cir_features, rgb_features], axis=1)
    elif cir:
        features = cir_features
    elif rgb:
        features = rgb_features
    else:
        print('No valid data input.')
    
    ground_truth = np.stack(gt_list, axis=0)

    features = torch.from_numpy(features)
    features = features.cuda()
    ground_truth = torch.from_numpy(ground_truth)
    
    ########################################################## 
    return features, ground_truth

In [ ]:
#putting the dataset into the TensorDataset wrapper
X, y = read_patch(training_set_path, use_cir, use_rgb)
X_t, y_t = read_patch(test_set_path, use_cir, use_rgb)

print(X.shape)
print(X_t.shape)

train_set = tnt.dataset.TensorDataset(list([X, y]))
test_set  = tnt.dataset.TensorDataset(list([X_t, y_t]))
print(len(train_set))

In [ ]:
class ConfusionMatrix:
    def __init__(self, n_class, class_names):
        self.CM = np.zeros((n_class, n_class))
        self.n_class = n_class
        self.class_names = class_names
  
    def clear(self):
        self.CM = np.zeros((self.n_class, self.n_class))
    
    def add_batch(self, gt, pred):
        self.CM +=  confusion_matrix(gt, pred, labels = list(range(self.n_class)))
    
    def overall_accuracy(self):#percentage of correct classification
        return 100*self.CM.trace() / self.CM.sum()

    def class_IoU(self, show = 1):
        ious = np.full(self.n_class, 0.)
        for i_class in range(self.n_class):
            ious[i_class] = self.CM[i_class, i_class] / \
                (-self.CM[i_class, i_class] \
                + self.CM[i_class, :].sum()
                + self.CM[:, i_class].sum())
        if show:
            print('  |  '.join('{} : {:3.2f}%'.format(name, 100*iou) for name, iou in zip(self.class_names,ious)))
        #do not count classes that are not present in the dataset in the mean IoU
        return 100*np.nansum(ious) / (np.logical_not(np.isnan(ious))).sum()

In [ ]:
m = ConfusionMatrix(3, class_names)
m.add_batch(np.array([0,1,1,1,2,0,0,2,0,2,1]), np.array([0,1,0,0,2,0,1,2,0,2,1]))
m.add_batch(np.array([0,1,2,1,2,1,0,2,1]), np.array([0,1,1,1,2,1,0,2,0]))
print(m.CM)
print("OA = %3.2f%%" % (m.overall_accuracy()))
m.class_IoU()
m.clear()

In [ ]:
class SegNet(nn.Module):
    """
    SegNet network for semantic segmentation
    """
  
    def __init__(self, n_channels, encoder_conv_width, decoder_conv_width, n_class, cuda = 1):
        """
        initialization function
        n_channels, int, number of input channel
        encoder_conv_width, int list, size of the feature maps of convs for the encoder
        decoder_conv_width, int list, size of the feature maps of convs for the decoder
        n_class = int,  the number of classes
        """
        super(SegNet, self).__init__() #necessary for all classes extending the module class
    
        assert((encoder_conv_width[3] == encoder_conv_width[5]) \
            and (encoder_conv_width[1] == decoder_conv_width[1]))
    
        self.maxpool=nn.MaxPool2d(2,2,return_indices=True) #maxpooling layer
        self.unpool=nn.MaxUnpool2d(2,2) #unpooling layer
    
        #encoder
        self.c1 = nn.Sequential(nn.Conv2d(n_channels,encoder_conv_width[0],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[0]),nn.ReLU(True))
        self.c2 = nn.Sequential(nn.Conv2d(encoder_conv_width[0],encoder_conv_width[1],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[1]),nn.ReLU(True))
        self.c3 = nn.Sequential(nn.Conv2d(encoder_conv_width[1],encoder_conv_width[2],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[2]),nn.ReLU(True))
        self.c4 = nn.Sequential(nn.Conv2d(encoder_conv_width[2],encoder_conv_width[3],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[3]),nn.ReLU(True))
        self.c5 = nn.Sequential(nn.Conv2d(encoder_conv_width[3],encoder_conv_width[4],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[4]),nn.ReLU(True))
        self.c6 = nn.Sequential(nn.Conv2d(encoder_conv_width[4],encoder_conv_width[5],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[5]),nn.ReLU(True))
        #decoder
        self.c7=nn.Sequential(nn.Conv2d(encoder_conv_width[5]+encoder_conv_width[3],decoder_conv_width[0],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(decoder_conv_width[0]),nn.ReLU(True))
        self.c8=nn.Sequential(nn.Conv2d(decoder_conv_width[0],decoder_conv_width[1],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(decoder_conv_width[1]),nn.ReLU(True))       
        self.c9=nn.Sequential(nn.Conv2d(encoder_conv_width[1] + decoder_conv_width[1],decoder_conv_width[2],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(decoder_conv_width[2]),nn.ReLU(True))
        self.c10=nn.Sequential(nn.Conv2d(decoder_conv_width[2],decoder_conv_width[3],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(decoder_conv_width[3]),nn.ReLU(True))
        #final classifying layer
        self.classifier=nn.Conv2d(decoder_conv_width[3],n_class,3,padding=1, padding_mode='reflect')

        #weight initialization

        self.c1[0].apply(self.init_weights)
        self.c2[0].apply(self.init_weights)
        self.c3[0].apply(self.init_weights)
        self.c4[0].apply(self.init_weights)
        self.c5[0].apply(self.init_weights)
        self.c6[0].apply(self.init_weights)
        self.c7[0].apply(self.init_weights)
        self.c8[0].apply(self.init_weights)
        self.c9[0].apply(self.init_weights)
        self.c10[0].apply(self.init_weights)
        self.classifier.apply(self.init_weights)
    
        if cuda: #put the model on the GPU memory
            self.cuda()
    
    def init_weights(self,layer): #gaussian init for the conv layers
        nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
    
    def forward(self,input):
        """
        the function called to run inference
        """  
        #encoder
        #level 1
        x1 = self.c2(self.c1(input))
        x2, indices_a_b =self.maxpool(x1)
        #level 2
        x3=self.c4(self.c3(x2))
        x4, indices_b_c =self.maxpool(x3)
        #level 3
        x5 = self.c6(self.c5(x4))
        #decoder
        #level 2       
        y4 = self.unpool(x5, indices_b_c)
        y3 = self.c8(self.c7(torch.cat((y4,x3),1)))
        #level 1       
        y2 = self.unpool(y3, indices_a_b)
        y1 = self.c10(self.c9(torch.cat((y2,x1),1)))
        #output         
        out = self.classifier(y1)
    
        return out
    
    """
    #encoder
    self.c1 = nn.Sequential(nn.Conv2d(n_channels,encoder_conv_width[0],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[0]),nn.ReLU(True))
    self.c2 = nn.Sequential(nn.Conv2d(encoder_conv_width[0],encoder_conv_width[1],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[1]),nn.ReLU(True))
    self.c3 = nn.Sequential(nn.Conv2d(encoder_conv_width[1],encoder_conv_width[2],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[2]),nn.ReLU(True))
    self.c4 = nn.Sequential(nn.Conv2d(encoder_conv_width[2],encoder_conv_width[3],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(encoder_conv_width[3]),nn.ReLU(True))
    
    self.c9=nn.Sequential(nn.Conv2d(encoder_conv_width[1] + encoder_conv_width[3],decoder_conv_width[0],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(decoder_conv_width[0]),nn.ReLU(True))
    self.c10=nn.Sequential(nn.Conv2d(decoder_conv_width[0],decoder_conv_width[1],3,padding=1, padding_mode='reflect'),nn.BatchNorm2d(decoder_conv_width[0]),nn.ReLU(True))
    #final classifying layer
    self.classifier=nn.Conv2d(decoder_conv_width[1],n_class,3,padding=1, padding_mode='reflect')

    #weight initialization

    self.c1[0].apply(self.init_weights)
    self.c2[0].apply(self.init_weights)
    self.c3[0].apply(self.init_weights)
    self.c4[0].apply(self.init_weights)
    self.c9[0].apply(self.init_weights)
    self.c10[0].apply(self.init_weights)
    self.classifier.apply(self.init_weights)
    
    if cuda: #put the model on the GPU memory
      self.cuda()
    
  def init_weights(self,layer): #gaussian init for the conv layers
    nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
    
  def forward(self,input):

    #encoder
    #level 1
    x1 = self.c2(self.c1(input))
    x2, indices_a_b =self.maxpool(x1)
    #level 2
    x3=self.c4(self.c3(x2))
    
    #decoder
    #level 1       
    y2 = self.unpool(x3, indices_a_b)
    y1 = self.c10(self.c9(torch.cat((y2,x1),1)))
    #output         
    out = self.classifier(y1)
    return out"""

In [ ]:
#we consider the first point cloud from the training set
tile, gt = read_patch(training_set_path)
print(tile.shape)
print(gt.shape)
print(tile[0:2,:,:,:].shape)
segnet = SegNet(3,[4,4,4,4], [4,4],3)
print(segnet)
print('Total number of parameters: {}'.format(sum([p.numel() for p in segnet.parameters()])))
pred = segnet(tile[0:2,:,:,:]) #the None indicate a batch size of 1
assert(pred.shape == torch.Size([2,3,500,500]))


In [ ]:
loader = torch.utils.data.DataLoader(train_set, \
         batch_size=args.batch_size, shuffle=True, drop_last=True)

for index, (tiles, gt) in enumerate(loader):
    print(index)
    print(tiles.shape)
    print(gt.shape)

In [ ]:
def train(model, optimizer, args):
  """train for one epoch"""
  model.train() #switch the model in training mode
  
  #the loader function will take care of the batching
  loader = torch.utils.data.DataLoader(train_set, \
         batch_size=args.batch_size, shuffle=True, drop_last=True)
  #tqdm will provide some nice progress bars
  loader = tqdm(loader, ncols=500)
  
  #will keep track of the loss
  loss_meter = tnt.meter.AverageValueMeter()
  cm = ConfusionMatrix(args.n_class, class_names = class_names)

  for index, (tiles, gt) in enumerate(loader):
    
    optimizer.zero_grad() #put gradient to zero
    
    pred = model(tiles) #compute the prediction

    loss = nn.functional.cross_entropy(pred.cpu(),gt, weight=torch.tensor([0.1,0.3,0.6]))

    loss.backward() #compute gradients

    for p in model.parameters(): #we clip the gradient at norm 1
      p.grad.data.clamp_(-1, 1) #this helps learning faster
    
    optimizer.step() #one SGD step
    
    loss_meter.add(loss.item())
    labeled = np.where(gt.view(-1)!=99)[0] #select gt with a label
    #need to put the prediction back on the cpu and convert to numpy
    cm.add_batch(gt.view(-1)[labeled], pred.argmax(1).view(-1)[labeled].cpu().detach().numpy())
    
  return cm, loss_meter.value()[0]

def eval(model, args):
  """eval on test/validation set"""
  
  model.eval() #switch in eval mode
  
  loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, drop_last=False)
  
  loader = tqdm(loader, ncols=500)
  
  loss_meter = tnt.meter.AverageValueMeter()
  cm = ConfusionMatrix(args.n_class, class_names = class_names)

  with torch.no_grad():
    for index, (tiles, gt) in enumerate(loader):
      
      #TODO
      pred = model(tiles)
      loss = nn.functional.cross_entropy(pred.cpu(),gt)
      loss_meter.add(loss.item())
      labeled = np.where(gt.view(-1)!=99)[0] #select gt with a label
      cm.add_batch(gt.view(-1)[labeled], pred.argmax(1).view(-1)[labeled].cpu().detach().numpy())

  return cm, loss_meter.value()[0]


def train_full(args):
  """The full training loop"""
  #initialize the model
  
  model = SegNet(args.n_channel, args.conv_width, args.dconv_width, args.n_class)

  print('Total number of parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
  
  #define the optimizer
  #adam optimizer is always a good guess for classification
  optimizer = optim.Adam(model.parameters(), lr=args.lr)
  scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[70,90], gamma=0.7)
  
  TESTCOLOR = '\033[104m'
  TRAINCOLOR = '\033[100m'
  NORMALCOLOR = '\033[0m'
  
  training_accu=[]
  training_miou=[]
  training_loss=[]
  training_epoch=[]
  test_accu=[]
  test_miou=[]
  test_loss=[]
  test_epoch=[]

  for i_epoch in range(args.n_epoch):
    #train one epoch
    cm_train, loss_train = train(model, optimizer, args)
    scheduler.step()
    print(TRAINCOLOR)
    print('Epoch %3d -> Train Overall Accuracy: %3.2f%% Train mIoU : %3.2f%% Train Loss: %1.4f' % (i_epoch, cm_train.overall_accuracy(), cm_train.class_IoU(), loss_train) + NORMALCOLOR)
    training_epoch.append(i_epoch)
    training_accu.append(cm_train.overall_accuracy())
    training_miou.append(cm_train.class_IoU())
    training_loss.append(loss_train)

    if (i_epoch == args.n_epoch - 1) or (args.n_epoch_test != 0 and i_epoch % args.n_epoch_test == 0 and i_epoch > 0):
      #periodic testing
      cm_test, loss_test = eval(model, args)
      print(TESTCOLOR)
      print('Test Overall Accuracy: %3.2f%% Test mIoU : %3.2f%%  Test Loss: %1.4f' % (cm_test.overall_accuracy(), cm_test.class_IoU(), loss_test) + NORMALCOLOR)
      test_epoch.append(i_epoch)
      test_accu.append(cm_test.overall_accuracy())
      test_miou.append(cm_test.class_IoU())
      test_loss.append(loss_test)
      #viewer(n_shown = 1, train = False, model = model, category = 'cigpe', use_mask = False)

  plt.figure(figsize=(30, 10))
  plt.subplot(1,3,1, ylim=(0,100), xlabel="# of epochs", ylabel="Overall Accuracy")
  plt.plot(training_epoch, training_accu)
  plt.plot(test_epoch, test_accu)
  plt.subplot(1,3,2, ylim=(0,100), xlabel="# of epochs", ylabel="mean Class IoU")
  plt.plot(training_epoch, training_miou)
  plt.plot(test_epoch, test_miou)
  plt.subplot(1,3,3, ylim=(0,1), xlabel="# of epochs", ylabel="Loss function")
  plt.plot(training_epoch, training_loss)
  plt.plot(test_epoch, test_loss)
  plt.show()
  return model

In [ ]:
def train(model, optimizer, args):
    """train for one epoch"""
    model.train() #switch the model in training mode
  
    #the loader function will take care of the batching
    loader = torch.utils.data.DataLoader(train_set, \
         batch_size=args.batch_size, shuffle=True, drop_last=True)
  
    #will keep track of the loss
    loss_meter = tnt.meter.AverageValueMeter()

    for index, (tiles, gt) in enumerate(loader):
    
        optimizer.zero_grad() #put gradient to zero
    
        pred = model(tiles) #compute the prediction

        loss = nn.functional.cross_entropy(pred.cpu(),gt)

        loss.backward() #compute gradients

        for p in model.parameters(): #we clip the gradient at norm 1
            p.grad.data.clamp_(-1, 1) #this helps learning faster
    
        optimizer.step() #one SGD step
    
        loss_meter.add(loss.item())
    
    return loss_meter.value()[0]

def eval(model, args):
    """eval on test/validation set"""
  
    model.eval() #switch in eval mode
  
    loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, drop_last=False)
  
    loader = tqdm(loader, ncols=500)
  
    loss_meter = tnt.meter.AverageValueMeter()
    cm = ConfusionMatrix(args.n_class, class_names = class_names)

    with torch.no_grad():
        for index, (tiles, gt) in enumerate(loader):
      
        #TODO
        pred = model(tiles)
        loss = nn.functional.cross_entropy(pred.cpu(),gt)
        loss_meter.add(loss.item())
        labeled = np.where(gt.view(-1)!=99)[0] #select gt with a label
        cm.add_batch(gt.view(-1)[labeled], pred.argmax(1).view(-1)[labeled].cpu().detach().numpy())

    return cm, loss_meter.value()[0]


def train_full(args):
    """The full training loop"""

    #initialize the model
    model = SegNet(args.n_channel, args.conv_width, args.dconv_width, args.n_class)

    print('Total number of parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
  
    #define the optimizer
    #adam optimizer is always a good guess for classification
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[70,90], gamma=0.7)
  
    TESTCOLOR = '\033[104m'
    NORMALCOLOR = '\033[0m'
  
    test_accu=[]
    test_miou=[]
    test_loss=[]
    test_epoch=[]

    for i_epoch in range(args.n_epoch):
        #train one epoch
        loss_train = train(model, optimizer, args)
        scheduler.step()

        if (i_epoch == args.n_epoch - 1) or (args.n_epoch_test != 0 and i_epoch % args.n_epoch_test == 0 and i_epoch > 0):
            #periodic testing
            cm_test, loss_test = eval(model, args)
            print(TESTCOLOR)
            print('Test Overall Accuracy: %3.2f%% Test mIoU : %3.2f%%  Test Loss: %1.4f' % (cm_test.overall_accuracy(), cm_test.class_IoU(), loss_test) + NORMALCOLOR)
            test_epoch.append(i_epoch)
            test_accu.append(cm_test.overall_accuracy())
            test_miou.append(cm_test.class_IoU())
            test_loss.append(loss_test)

    plt.figure(figsize=(30, 10))
    plt.subplot(1,3,1, ylim=(0,100), xlabel="# of epochs", ylabel="Overall Accuracy")
    plt.plot(test_epoch, test_accu)
    plt.subplot(1,3,2, ylim=(0,100), xlabel="# of epochs", ylabel="mean Class IoU")
    plt.plot(test_epoch, test_miou)
    plt.subplot(1,3,3, ylim=(0,1), xlabel="# of epochs", ylabel="Loss function")
    plt.plot(test_epoch, test_loss)
    plt.show()
    return model

In [ ]:
args = mock.Mock() #stores the parameters
args.n_epoch = 50
args.n_epoch_test = int(5) #periodicity of evaluation on test set
args.batch_size = 4
args.n_class = len(class_names)
args.n_channel = 6 if use_cir and use_rgb else 3
args.conv_width = [8,16,8,8,8,8]
args.dconv_width = [8,16,8,8]
args.cuda = 1
args.lr = 1e-3

a = time()
trained_model = train_full(args)
b = time()

print('Training finished in ' + str(b-a) + 's')

In [ ]:
print(X_t.shape)
print(y_t.shape)

print(X_t.max())
print(X_t.min())
print(y_t.max())
print(y_t.min())

In [ ]:
def plot_rgb_cir_gt_pred(tile_index, data, gt, model, cir, rgb):
    # Function to plot prediction vs ground truth
    
    # Plotting
    plt.figure(facecolor='white')

    data = data[tile_index,:,:,:]
    pred = model(data[None,:,:,:]).cpu().detach().numpy()
    pred = pred[0,:,:,:].argmax(0).squeeze()
    
    unique, counts = np.unique(pred, return_counts=True)
    print(dict(zip(unique, counts)))
    
    data = data.cpu().numpy().astype(np.uint8)
    
    if cir and rgb:
        plt.subplot(1, 4, 1)
        plt.imshow(data[:3].transpose([1,2,0]))
        plt.title('NIR Red Green composite')
        plt.axis('off')

        plt.subplot(1, 4, 2)
        plt.imshow(data[-3:].transpose([1,2,0]))
        plt.title('Red Green Blue composite')
        plt.axis('off')
        
        plt.subplot(1, 4, 3)
        plt.imshow(gt[tile_index,:,:], CMAP)
        plt.title('GT Labels')
        plt.axis('off')

        plt.subplot(1, 4, 4)
        plt.imshow(pred, CMAP)
        plt.title('Predicted Labels')
        plt.axis('off')
    
    elif cir or rgb:
        plt.subplot(1, 3, 1)
        plt.imshow(data.transpose([1,2,0]))
        if cir:
            plt.title('NIR Red Green composite')
        else:
            plt.title('Red Green Blue composite')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(gt[tile_index,:,:], CMAP)
        plt.title('GT Labels')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(pred, CMAP)
        plt.title('Predicted Labels')
        plt.axis('off')

In [ ]:
plot_rgb_cir_gt_pred(3, X_t, y_t, trained_model, use_cir, use_rgb)